In [1]:
import sys, random
import numpy as np
import pickle
from collections import OrderedDict
import argparse

In [2]:
import theano
import theano.tensor as T
from theano import config

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
import tensorflow as tf

In [4]:
def numpy_floatX(data):
	return np.asarray(data, dtype=config.floatX)

def unzip(zipped):
	new_params = OrderedDict()
	for k, v in zipped.items():
		new_params[k] = v.get_value()
	return new_params

In [5]:
def init_params(options):
	params = OrderedDict()

	numXcodes = options['numXcodes']
	numYcodes = options['numYcodes']
	embDimSize= options['embDimSize']
	demoSize = options['demoSize']
	hiddenDimSize = options['hiddenDimSize']

	params['W_emb'] = np.random.uniform(-0.01, 0.01, (numXcodes, embDimSize)).astype(config.floatX) #emb matrix needs an extra dimension for the time
	params['b_emb'] = np.zeros(embDimSize).astype(config.floatX)
	params['W_hidden'] = np.random.uniform(-0.01, 0.01, (embDimSize+demoSize, hiddenDimSize)).astype(config.floatX) #emb matrix needs an extra dimension for the time
	params['b_hidden'] = np.zeros(hiddenDimSize).astype(config.floatX)
	if numYcodes > 0:
		params['W_output'] = np.random.uniform(-0.01, 0.01, (hiddenDimSize, numYcodes)).astype(config.floatX) #emb matrix needs an extra dimension for the time
		params['b_output'] = np.zeros(numYcodes).astype(config.floatX)
	else:
		params['W_output'] = np.random.uniform(-0.01, 0.01, (hiddenDimSize, numXcodes)).astype(config.floatX) #emb matrix needs an extra dimension for the time
		params['b_output'] = np.zeros(numXcodes).astype(config.floatX)

	return params

#這邊出來會是一個dict

In [6]:
def load_params(options):
	params = np.load(options['modelFile'])
	return params

def init_tparams(params):
	tparams = OrderedDict()
	for k, v in params.items():
		tparams[k] = theano.shared(v, name=k)
	return tparams
#這邊用了一個theano是甚麼我也不知道 反正好像還是一個dict
#造成在下面跑T.grad會出錯 (T=theano)
#tparams 是一個...dict 但他的數值好像被藏起來了

In [76]:
theano.config.compute_test_value = "warn"

In [77]:
def get_tparam_keys(tparams):
    ls=[]
    for k in tparams.keys():
        ls.append(tparams[k].get_value())
    return ls

In [78]:
#好像是你第一個cost不能填數字0?
#你知道這邊的cost該填什麼嗎?
#自問自答 這邊放的值要用theano的格式(也就是可以儲存梯度)
T.grad(T.log(1),wrt=[*init_tparams(params).values()])

NameError: name 'params' is not defined

In [ ]:
init_tparams(params).values()

In [ ]:
#嘎嘎嘎 恐龍在吃程式碼 嘎嘎嘎 程式無法執行

In [7]:
def build_model(tparams, options):
	x = T.matrix('x', dtype=config.floatX)
	d = T.matrix('d', dtype=config.floatX)
	y = T.matrix('y', dtype=config.floatX)
	mask = T.vector('mask', dtype=config.floatX)

	logEps = options['logEps']

	emb = T.maximum(T.dot(x, tparams['W_emb']) + tparams['b_emb'],0)
	if options['demoSize'] > 0: emb = T.concatenate((emb, d), axis=1)
	visit = T.maximum(T.dot(emb, tparams['W_hidden']) + tparams['b_hidden'],0)
	results = T.nnet.softmax(T.dot(visit, tparams['W_output']) + tparams['b_output'])
	
	mask1 = (mask[:-1] * mask[1:])[:,None]
	mask2 = (mask[:-2] * mask[1:-1] * mask[2:])[:,None]
	mask3 = (mask[:-3] * mask[1:-2] * mask[2:-1] * mask[3:])[:,None]
	mask4 = (mask[:-4] * mask[1:-3] * mask[2:-2] * mask[3:-1] * mask[4:])[:,None]
	mask5 = (mask[:-5] * mask[1:-4] * mask[2:-3] * mask[3:-2] * mask[4:-1] * mask[5:])[:,None]

	t = None
	if options['numYcodes'] > 0: t = y
	else: t = x

	forward_results =  results[:-1] * mask1
	forward_cross_entropy = -(t[1:] * T.log(forward_results + logEps) + (1. - t[1:]) * T.log(1. - forward_results + logEps))

	forward_results2 =  results[:-2] * mask2
	forward_cross_entropy2 = -(t[2:] * T.log(forward_results2 + logEps) + (1. - t[2:]) * T.log(1. - forward_results2 + logEps))

	forward_results3 =  results[:-3] * mask3
	forward_cross_entropy3 = -(t[3:] * T.log(forward_results3 + logEps) + (1. - t[3:]) * T.log(1. - forward_results3 + logEps))

	forward_results4 =  results[:-4] * mask4
	forward_cross_entropy4 = -(t[4:] * T.log(forward_results4 + logEps) + (1. - t[4:]) * T.log(1. - forward_results4 + logEps))

	forward_results5 =  results[:-5] * mask5
	forward_cross_entropy5 = -(t[5:] * T.log(forward_results5 + logEps) + (1. - t[5:]) * T.log(1. - forward_results5 + logEps))

	backward_results =  results[1:] * mask1
	backward_cross_entropy = -(t[:-1] * T.log(backward_results + logEps) + (1. - t[:-1]) * T.log(1. - backward_results + logEps))

	backward_results2 =  results[2:] * mask2
	backward_cross_entropy2 = -(t[:-2] * T.log(backward_results2 + logEps) + (1. - t[:-2]) * T.log(1. - backward_results2 + logEps))

	backward_results3 =  results[3:] * mask3
	backward_cross_entropy3 = -(t[:-3] * T.log(backward_results3 + logEps) + (1. - t[:-3]) * T.log(1. - backward_results3 + logEps))

	backward_results4 =  results[4:] * mask4
	backward_cross_entropy4 = -(t[:-4] * T.log(backward_results4 + logEps) + (1. - t[:-4]) * T.log(1. - backward_results4 + logEps))

	backward_results5 =  results[5:] * mask5
	backward_cross_entropy5 = -(t[:-5] * T.log(backward_results5 + logEps) + (1. - t[:-5]) * T.log(1. - backward_results5 + logEps))

	visit_cost1 = (forward_cross_entropy.sum(axis=1).sum(axis=0) + backward_cross_entropy.sum(axis=1).sum(axis=0)) / (mask1.sum() + logEps)
	visit_cost2 = (forward_cross_entropy2.sum(axis=1).sum(axis=0) + backward_cross_entropy2.sum(axis=1).sum(axis=0)) / (mask2.sum() + logEps)
	visit_cost3 = (forward_cross_entropy3.sum(axis=1).sum(axis=0) + backward_cross_entropy3.sum(axis=1).sum(axis=0)) / (mask3.sum() + logEps)
	visit_cost4 = (forward_cross_entropy4.sum(axis=1).sum(axis=0) + backward_cross_entropy4.sum(axis=1).sum(axis=0)) / (mask4.sum() + logEps)
	visit_cost5 = (forward_cross_entropy5.sum(axis=1).sum(axis=0) + backward_cross_entropy5.sum(axis=1).sum(axis=0)) / (mask5.sum() + logEps)

	windowSize = options['windowSize']
	visit_cost = visit_cost1
	if windowSize == 2:
		visit_cost = visit_cost1 + visit_cost2
	elif windowSize == 3:
		visit_cost = visit_cost1 + visit_cost2 + visit_cost3
	elif windowSize == 4:
		visit_cost = visit_cost1 + visit_cost2 + visit_cost3 + visit_cost4
	elif windowSize == 5:
		visit_cost = visit_cost1 + visit_cost2 + visit_cost3 + visit_cost4 + visit_cost5

	iVector = T.vector('iVector', dtype='int32')
	jVector = T.vector('jVector', dtype='int32')
	preVec = T.maximum(tparams['W_emb'],0)
	norms = (T.exp(T.dot(preVec, preVec.T))).sum(axis=1)
	emb_cost = -T.log((T.exp((preVec[iVector] * preVec[jVector]).sum(axis=1)) / norms[iVector]) + logEps)
    
	#print(visit_cost,T.mean(emb_cost),options["L2_reg"],tparams["W_emb"])
    
	total_cost = visit_cost + T.mean(emb_cost) + options['L2_reg'] * (tparams['W_emb'] ** 2).sum()

	if options['demoSize'] > 0 and options['numYcodes'] > 0: return x, d, y, mask, iVector, jVector, total_cost
	elif options['demoSize'] == 0 and options['numYcodes'] > 0: return x, y, mask, iVector, jVector, total_cost
	elif options['demoSize'] > 0 and options['numYcodes'] == 0: return x, d, mask, iVector, jVector, total_cost
	else: return x, mask, iVector, jVector, total_cost



In [8]:
def adadelta(tparams, grads, x, mask, iVector, jVector, cost, options, d=None, y=None):
	zipped_grads = [theano.shared(p.get_value() * numpy_floatX(0.), name='%s_grad' % k) for k, p in tparams.items()]
	running_up2 = [theano.shared(p.get_value() * numpy_floatX(0.), name='%s_rup2' % k) for k, p in tparams.items()]
	running_grads2 = [theano.shared(p.get_value() * numpy_floatX(0.), name='%s_rgrad2' % k) for k, p in tparams.items()]

	zgup = [(zg, g) for zg, g in zip(zipped_grads, grads)]
	rg2up = [(rg2, 0.95 * rg2 + 0.05 * (g ** 2)) for rg2, g in zip(running_grads2, grads)]

	if options['demoSize'] > 0 and options['numYcodes'] > 0:
		f_grad_shared = theano.function([x, d, y, mask, iVector, jVector], cost, updates=zgup + rg2up, name='adadelta_f_grad_shared')
	elif options['demoSize'] == 0 and options['numYcodes'] > 0:
		f_grad_shared = theano.function([x, y, mask, iVector, jVector], cost, updates=zgup + rg2up, name='adadelta_f_grad_shared')
	elif options['demoSize'] > 0 and options['numYcodes'] == 0:
		f_grad_shared = theano.function([x, d, mask, iVector, jVector], cost, updates=zgup + rg2up, name='adadelta_f_grad_shared')
	else:
		f_grad_shared = theano.function([x, mask, iVector, jVector], cost, updates=zgup + rg2up, name='adadelta_f_grad_shared')

	updir = [-T.sqrt(ru2 + 1e-6) / T.sqrt(rg2 + 1e-6) * zg for zg, ru2, rg2 in zip(zipped_grads, running_up2, running_grads2)]
	ru2up = [(ru2, 0.95 * ru2 + 0.05 * (ud ** 2)) for ru2, ud in zip(running_up2, updir)]
	param_up = [(p, p + ud) for p, ud in zip(tparams.values(), updir)]

	f_update = theano.function([], [], updates=ru2up + param_up, on_unused_input='ignore', name='adadelta_f_update')

	return f_grad_shared, f_update

In [9]:
def load_data(xFile, dFile, yFile):
	seqX = np.array(pickle.load(open(xFile, 'rb')))
	seqD = []
	if len(dFile) > 0: seqD = np.asarray(pickle.load(open(dFile, 'rb')), dtype=config.floatX)
	seqY = []
	if len(yFile) > 0: seqY = np.array(pickle.load(open(yFile, 'rb')))
	return seqX, seqD, seqY

def pickTwo(codes, iVector, jVector):
	for first in codes:
		for second in codes:
			if first == second: continue
			iVector.append(first)
			jVector.append(second)

In [10]:
def padMatrix(seqs, labels, options):
	n_samples = len(seqs)
	iVector = []
	jVector = []
	numXcodes = options['numXcodes']
	numYcodes = options['numYcodes']

	if numYcodes > 0:
		x = np.zeros((n_samples, numXcodes)).astype(config.floatX)
		y = np.zeros((n_samples, numYcodes)).astype(config.floatX)
		mask = np.zeros((n_samples,)).astype(config.floatX)
#		for idx, (seq, label) in enumerate(zip(seqs, labels[0].values())): #sample file
		for idx, (seq, label) in enumerate(zip(seqs, labels)): #normal code
			if not seq[0] == -1:
				x[idx][seq] = 1.
				y[idx][label] = 1.
				pickTwo(seq, iVector, jVector)
				mask[idx] = 1.
		return x, y, mask, iVector, jVector
	else:
		x = np.zeros((n_samples, numXcodes)).astype(config.floatX)
		mask = np.zeros((n_samples,)).astype(config.floatX)
		for idx, seq in enumerate(seqs):
			if not seq[0] == -1:
				x[idx][seq] = 1.
				pickTwo(seq, iVector, jVector)
				mask[idx] = 1.
		return x, mask, iVector, jVector

In [36]:
def train_med2vec(seqFile='seqFile.txt', 
				demoFile='demoFile.txt',
				labelFile='labelFile.txt',
				outFile='outFile.txt',
				modelFile='modelFile.txt',
				L2_reg=0.001,
				numXcodes=20000, 
				numYcodes=20000, 
				embDimSize=1000,
				hiddenDimSize=2000,
				batchSize=100,
				demoSize=2,
				logEps=1e-8,
				windowSize=1,
				verbose=False,
				maxEpochs=1000):

	options = locals().copy()
	print('initializing parameters')
	params = init_params(options)
	#params = load_params(options)
	tparams = init_tparams(params)

	print('building models')
	f_grad_shared = None
	f_update = None
	#print(demoSize,numYcodes)
	if demoSize > 0 and numYcodes > 0:
		x, d, y, mask, iVector, jVector, cost = build_model(tparams, options)
		grads = T.grad(cost, wrt=[*tparams.values()])
        #grads = T.grad(cost, wrt=tparams.values())
		f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options, d=d, y=y)
	elif demoSize == 0 and numYcodes > 0:
		x, y, mask, iVector, jVector, cost = build_model(tparams, options)
		#print(cost)
		grads = T.grad(cost, wrt=[*tparams.values()])
        #grads = T.grad(cost, wrt=tparams.values())
		f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options, y=y)
	elif demoSize > 0 and numYcodes == 0:
		x, d, mask, iVector, jVector, cost = build_model(tparams, options)
		grads = T.grad(cost, wrt=[*tparams.values()])
		#grads = T.grad(cost, wrt=tparams.values())
		f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options, d=d)
	else:
		x, mask, iVector, jVector, cost = build_model(tparams, options)
		grads = list(T.grad(cost, wrt=[*tparams.values()]))
		#grads = T.grad(cost, wrt=tparams.values())
		f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options)

	print('loading data')
	seqs, demos, labels = load_data(seqFile, demoFile, labelFile)
	n_batches = int(np.ceil(float(len(seqs)) / float(batchSize)))

	print('training start')
	for epoch in range(maxEpochs):
		iteration = 0
		costVector = []
		for index in random.sample(range(n_batches), n_batches):
			batchX = seqs[batchSize*index:batchSize*(index+1)]
			batchY = []
			batchD = []
			label_length=len(labels.tolist())
			if demoSize > 0 and numYcodes > 0:
				#batchY = labels[batchSize*index:batchSize*(index+1)]
				#batchY = [labels.tolist()[batchSize*index:batchSize*(index+1)]]
				if batchSize*(index+1)>label_length:
					batchY = list(labels.tolist().values())[batchSize*index%label_length:]+list(labels.tolist().values())[:batchSize*(index+1)%label_length]
				else:
					batchY = list(labels.tolist().values())[batchSize*index%label_length:batchSize*(index+1)%label_length]
				x, y, mask, iVector, jVector = padMatrix(batchX, batchY, options)
				batchD = demos[batchSize*index:batchSize*(index+1)]
				cost = f_grad_shared(x, batchD, y, mask, iVector, jVector)
			elif demoSize == 0 and numYcodes > 0:
				#batchY = labels[batchSize*index:batchSize*(index+1)]
				if batchSize*(index+1)>label_length:
					batchY = list(labels.tolist().values())[batchSize*index%label_length:]+list(labels.tolist().values())[:batchSize*(index+1)%label_length]
				else:
					batchY = list(labels.tolist().values())[batchSize*index%label_length:batchSize*(index+1)%label_length]
				x, y, mask, iVector, jVector = padMatrix(batchX, batchY, options)
				cost = f_grad_shared(x, y, mask, iVector, jVector)
			elif demoSize > 0 and numYcodes == 0:
				x, mask, iVector, jVector = padMatrix(batchX, batchY, options)
				batchD = demos[batchSize*index:batchSize*(index+1)]
				cost = f_grad_shared(x, batchD, mask, iVector, jVector)
			else:
				x, mask, iVector, jVector = padMatrix(batchX, batchY, options)
				cost = f_grad_shared(x, mask, iVector, jVector)
			costVector.append(cost)
			f_update()
			if (iteration % 10 == 0) and verbose: print('epoch:%d, iteration:%d/%d, cost:%f' % (epoch, iteration, n_batches, cost))
			iteration += 1
		print('epoch:%d, mean_cost:%f' % (epoch, np.mean(costVector)))
		tempParams = unzip(tparams)
		np.savez_compressed(outFile + '.' + str(epoch), **tempParams)

In [ ]:
train_med2vec(seqFile='./mimic_demo/ourR301_100.seqs',
              demoFile='',
              labelFile='./mimic_demo/ourR301_100.types', 
              outFile='./mimic_demo/ourR301_100result.txt',
              numXcodes=20000,
              numYcodes=20000, 
              embDimSize=1000, 
              hiddenDimSize=2000, 
              batchSize=100, 
              maxEpochs=1000,
              L2_reg=0.001,
              demoSize=0,
              windowSize=1,
              logEps=1e-8,
              verbose='store_true')

initializing parameters
building models
loading data
training start
epoch:0, iteration:0/132, cost:32.376013
epoch:0, iteration:10/132, cost:32.709517
epoch:0, iteration:20/132, cost:33.034868
epoch:0, iteration:30/132, cost:32.946498
epoch:0, iteration:40/132, cost:31.198795
epoch:0, iteration:50/132, cost:27.038865
epoch:0, iteration:60/132, cost:27.816311
epoch:0, iteration:70/132, cost:27.477606
epoch:0, iteration:80/132, cost:25.467708
epoch:0, iteration:90/132, cost:24.798857
epoch:0, iteration:100/132, cost:25.031657
epoch:0, iteration:110/132, cost:26.545153
epoch:0, iteration:120/132, cost:25.868930
epoch:0, iteration:130/132, cost:25.744150
epoch:0, mean_cost:28.233457
epoch:1, iteration:0/132, cost:25.045388
epoch:1, iteration:10/132, cost:24.879817
epoch:1, iteration:20/132, cost:23.803209
epoch:1, iteration:30/132, cost:23.509776
epoch:1, iteration:40/132, cost:24.640266
epoch:1, iteration:50/132, cost:26.337807
epoch:1, iteration:60/132, cost:24.569578
epoch:1, iteration:

In [96]:
seqs, demos, labels = load_data('./mimic_demo/ourR301_100.seqs','','./mimic_demo/ourR301_100.types')

In [105]:
options={"numXcodes":20000,
        "numYcodes":20000, 
        "embDimSize":1000, 
        "hiddenDimSize":2000, 
        "batchSize":100, 
        "maxEpochs":1000,
        "L2_reg":0.001,
        "demoSize":0,
        "windowSize":1,
        "logEps":1e-8,
        "verbose":'store_true'}

In [106]:
params = init_params(options)

In [107]:
tparams = init_tparams(params)

In [117]:
f_grad_shared = None
f_update = None
#print(demoSize,numYcodes)
demoSize = options["demoSize"]
numYcodes = options["numYcodes"]
if demoSize > 0 and numYcodes > 0:
    x, d, y, mask, iVector, jVector, cost = build_model(tparams, options)
    grads = T.grad(cost, wrt=[*tparams.values()])
    #grads = T.grad(cost, wrt=tparams.values())
    f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options, d=d, y=y)
elif demoSize == 0 and numYcodes > 0:
    x, y, mask, iVector, jVector, cost = build_model(tparams, options)
    #print(cost)
    grads = T.grad(cost, wrt=[*tparams.values()])
    #grads = T.grad(cost, wrt=tparams.values())
    f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options, y=y)
elif demoSize > 0 and numYcodes == 0:
    x, d, mask, iVector, jVector, cost = build_model(tparams, options)
    grads = T.grad(cost, wrt=[*tparams.values()])
    #grads = T.grad(cost, wrt=tparams.values())
    f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options, d=d)
else:
    x, mask, iVector, jVector, cost = build_model(tparams, options)
    grads = list(T.grad(cost, wrt=[*tparams.values()]))
    #grads = T.grad(cost, wrt=tparams.values())
    f_grad_shared, f_update = adadelta(tparams, grads, x, mask, iVector, jVector, cost, options)

/tmp/ipykernel_479807/1114053983.py:9: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{add,no_inplace}.0) of Op Elemwise{maximum,no_inplace}(Elemwise{add,no_inplace}.0, InplaceDimShuffle{x,x}.0) missing default value
  emb = T.maximum(T.dot(x, tparams['W_emb']) + tparams['b_emb'],0)
/tmp/ipykernel_479807/1114053983.py:11: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{add,no_inplace}.0) of Op Elemwise{maximum,no_inplace}(Elemwise{add,no_inplace}.0, InplaceDimShuffle{x,x}.0) missing default value
  visit = T.maximum(T.dot(emb, tparams['W_hidden']) + tparams['b_hidden'],0)
/tmp/ipykernel_479807/1114053983.py:25: UserWarning: Warning, Cannot compute test value: input 0 (Elemwise{add,no_inplace}.0) of Op Elemwise{log,no_inplace}(Elemwise{add,no_inplace}.0) missing default value
  forward_cross_entropy = -(t[1:] * T.log(forward_results + logEps) + (1. - t[1:]) * T.log(1. - forward_results + logEps))
/tmp/ipykernel_479807/1114053983.py:28: UserWarning: 

In [128]:
batchSize = options["batchSize"]

In [18]:
print('loading data')
seqs, demos, labels = load_data('./mimic_demo/ourR301_100.seqs','','./mimic_demo/ourR301_100.types')
#n_batches = int(np.ceil(float(len(seqs)) / float(batchSize)))

loading data


In [19]:
len(seqs)

13154

In [22]:
len(labels.tolist())

430

In [23]:
len(seqs)/len(labels.tolist())

30.590697674418603

In [130]:
maxEpochs = options["maxEpochs"]

In [145]:
type(seqs)

numpy.ndarray

In [146]:
type(labels)

numpy.ndarray

In [147]:
seqs

array([list([0, 1, 2]), list([3, 4, 5]), list([6, 1, 7]), ...,
       list([269, 235, 7]), list([269, 235, 7]), list([269, 235, 7])],
      dtype=object)

In [206]:
batchY = list(labels.tolist().values())[batchSize*index%label_length:batchSize*(index+1)%label_length]

In [208]:
print('training start')
for epoch in range(maxEpochs):
    iteration = 0
    costVector = []
    for index in random.sample(range(n_batches), n_batches):
        batchX = seqs[batchSize*index:batchSize*(index+1)]
        batchY = []
        batchD = []
        label_length=len(labels.tolist())
        if demoSize > 0 and numYcodes > 0:
            #batchY = labels[batchSize*index:batchSize*(index+1)]
            #batchY = [labels.tolist()[batchSize*index:batchSize*(index+1)]]
            batchY = list(labels.tolist().values())[batchSize*index:batchSize*(index+1)]
            if len(batchY)==0:
                pass
            else:
                x, y, mask, iVector, jVector = padMatrix(batchX, batchY, options)
                batchD = demos[batchSize*index:batchSize*(index+1)]
                cost = f_grad_shared(x, batchD, y, mask, iVector, jVector)
        elif demoSize == 0 and numYcodes > 0:
            #batchY = labels[batchSize*index:batchSize*(index+1)]
            batchY = list(labels.tolist().values())[batchSize*index:batchSize*(index+1)]
            if len(batchY)==0:
                pass
            else:
                x, y, mask, iVector, jVector = padMatrix(batchX, batchY, options)
                cost = f_grad_shared(x, y, mask, iVector, jVector)
        elif demoSize > 0 and numYcodes == 0:
            x, mask, iVector, jVector = padMatrix(batchX, batchY, options)
            batchD = demos[batchSize*index:batchSize*(index+1)]
            cost = f_grad_shared(x, batchD, mask, iVector, jVector)
        else:
            x, mask, iVector, jVector = padMatrix(batchX, batchY, options)
            cost = f_grad_shared(x, mask, iVector, jVector)
        costVector.append(cost)
        f_update()
        if (iteration % 10 == 0) and verbose: print('epoch:%d, iteration:%d/%d, cost:%f' % (epoch, iteration, n_batches, cost))
        iteration += 1

training start
epoch:0, iteration:0/132, cost:26.751900
epoch:0, iteration:10/132, cost:26.809044


KeyboardInterrupt: 

In [24]:
n_batches = int(np.ceil(float(len(seqs)) / float(100)))

In [30]:
for index in random.sample(range(n_batches), n_batches):
    #print(seqs[100*index:100*(index+1)])
    print(list(labels.tolist().values())[100*index:100*(index+1)])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122